### Запуск QtLab

In [1]:
%run D:\qtlab_replacement\init.py
    
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *
%matplotlib

root: WARNING  Unable to load config file.


Failed loading module Signal_Hound_SA:  [WinError 126] The specified module could not be found
Using matplotlib backend: TkAgg


In [6]:
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
pna = Agilent_N5242A('pna', address = 'PNA')
lo1 = Agilent_E8257D('lo1', address = 'PSG1')
current = Yokogawa_GS210(address='GPIB0::2::INSTR')

# Поиск резонаторов

In [3]:
#Параметры кубитов.
Qubits = {1: {'F': 6.87549e9,'F01': 7.4178e9,'I0': 0,'Ispan': 1.6e-3,'P': -63,'Pex': -16,'Span':25e6}}
#Qubits = {1: {'F': 3.175e9,'F01': 0.,'I0': 12.5e-5,'Ispan': 400e-6,'P': -40,'Pex': 18,'Span': 50e9}}
#Qubits = {1: {'F': 3.176453e9,'F01': 0.,'I0': 6.7e-5,'Ispan': 200e-6,'P': -40,'Pex': -5,'Span': 40e6}}
#Qubits = {1: {'F': 7.603e9,'F01': 8.11962e9,'I0': 35.2e-6,'Ispan': 0.,'P': -65,'Pex': -18.,'Span': 20e6}}

In [3]:
lo1.set_power(16)

True

In [ ]:
#pna.set_timeout(30000)

In [4]:
import imp
import save_pkl
imp.reload(save_pkl)
pna.set_xlim(9.4315e9, 9.4315e9)
pna.set_nop(1)
pna.set_bandwidth(10)
lo1.set_power(0)
pna.set_power(-55)


True

In [12]:
arr=np.linspace(5.4e9,6.4e9,501)
lo1.set_status(1)
arrpower=np.linspace(-20, 10, 11)

sweep.sweep(pna, (arrpower, lo1.set_power, 'Power'), (arr, lo1.set_frequency, 'Pump_frequency'), filename='2nd tone spectroscopy2')


Started at:  Oct 18 2017 11:23:01


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Power:     10,	Pump_frequency: 6.4e+09,	average cycle time: 0.58s	8s			
Elapsed time: 0 h 53 m 20.85 s


{'S-parameter': (('Power', 'Pump_frequency'),
  (array([-20., -17., -14., -11.,  -8.,  -5.,  -2.,   1.,   4.,   7.,  10.]),
   array([  5.40000000e+09,   5.40200000e+09,   5.40400000e+09,
            5.40600000e+09,   5.40800000e+09,   5.41000000e+09,
            5.41200000e+09,   5.41400000e+09,   5.41600000e+09,
            5.41800000e+09,   5.42000000e+09,   5.42200000e+09,
            5.42400000e+09,   5.42600000e+09,   5.42800000e+09,
            5.43000000e+09,   5.43200000e+09,   5.43400000e+09,
            5.43600000e+09,   5.43800000e+09,   5.44000000e+09,
            5.44200000e+09,   5.44400000e+09,   5.44600000e+09,
            5.44800000e+09,   5.45000000e+09,   5.45200000e+09,
            5.45400000e+09,   5.45600000e+09,   5.45800000e+09,
            5.46000000e+09,   5.46200000e+09,   5.46400000e+09,
            5.46600000e+09,   5.46800000e+09,   5.47000000e+09,
            5.47200000e+09,   5.47400000e+09,   5.47600000e+09,
            5.47800000e+09,   5.48000000e+09

In [6]:
%matplotlib

Using matplotlib backend: TkAgg


In [93]:

ind =1

lo1.set_status(0)

pna.set_sweep_mode("LIN")
pna.set_nop(200)
pna.set_centerfreq(Qubits[ind]['F'])
pna.set_span(Qubits[ind]['Span'])
pna.set_bandwidth(10)
pna.set_power(-64)
pna.set_average(0)
pna.set_averages (1)
try:
    #pass
    current.set_status(1)
    #current.set_autorange(1)
    current.set_current(Qubits[ind]['I0'])
except:
    print ("no current")
    pass

#pna.set_timeout()

res_data = pna.measure()["S-parameter"]
res_freqs = pna.get_freqpoints()

plt.figure("S21_amp")
plt.plot(res_freqs, abs(res_data))
plt.figure("S21_ph")
plt.plot(res_freqs[1:], np.diff(np.unwrap(np.angle(res_data))))


In [19]:
#Save
exec(open('save_pkl.py').read())
save_pkl_noplt('res_pna_-30_250K', {"Amp": res_data[0] , "Ph": res_data[1], "freqs": res_freqs } )

FileNotFoundError: [Errno 2] No such file or directory: 'save_pkl.py'

# Plotting

In [ ]:
res_data_ref = res_data

In [ ]:
plt.figure()
plt.plot(res_freqs, res_data[0] - res_data_ref[0])

In [ ]:
import pickle
infiles = ['C:/qtlab/ReiData/data/Apr 03 2017/res_pna_-30.pkl','C:/qtlab/ReiData/data/Apr 04 2017/res_pna_-30_250K.pkl']
for infile in infiles:
    with open(infile) as f:
        data = pickle.load(f)
        plt.figure('Amp')
        plt.plot(data['freqs'], np.log10(data['Amp'])*20, label=infile)
        plt.figure('Ph')
        plt.plot(data['freqs'], data['Ph'], label=infile)
plt.figure('Amp')
plt.legend()
plt.figure('Ph')
plt.legend()

# Одномерный скан по полю

In [ ]:
#q_to_meas = Qubits.keys()
#Померить не все
q_to_meas = [1,]
#Шаг по втотрому тону
useIspan = False
cur_step = 0.5e-6
current.set_current(-240e-6)
#currents = np.linspace(-3e-4, -1e-4, 100)

pna.set_nop(200)
#header= "#Parameter = current"
lo1.set_status(0)
current.set_status(1)
#current.set_autorange(1)
pna.set_bandwidth(500)
#root_dir = dm.mk_dir()

for q_id in q_to_meas:
    if useIspan:
        pna.set_power(Qubits[q_id]['P'])
        Ipoints = np.arange( -Qubits[q_id]['Ispan']/2., Qubits[q_id]['Ispan']/2., cur_step)
        currents = Ipoints + Qubits[q_id]['I0']
    pna.set_nop(1)
    pna.set_centerfreq(Qubits[q_id]['F'])
 #   end_dir = root_dir+"{:d}/".format(q_id)
 #   dm.mk_dir(end_dir)
    try:
        sweep.sweep(pna, (currents, current.set_current, 'Coil current'),
            filename='Acouston_Isw_p_-30_1')
    except KeyboardInterrupt: break

In [ ]:
    try:
        sweep.sweep(pna, (currents, current.set_current, 'Coil current'),
            filename='Acouston_Isw_p_-30_1')
    except KeyboardInterrupt: break

In [ ]:
# current.set_current(0)

In [ ]:
(2.4e-4-1.06e-4)/2.

# Двумерный скан по полю

In [31]:
#q_to_meas = Qubits.keys()
#Померить не все
q_to_meas = [ 1,]
currents = np.linspace(3.7e-5, 5.4e-5, 401)
#currents = np.linspace(-5.2e-5, -4.2e-5, 201) - anticri
#currents = np.linspace( -1e-3, 1e-3, 300)
#powers = np.linspace( -30, 15, 9)

current.set_status(1)
#current.set_autorange(1)

pna.set_nop(401)
pna.set_bandwidth(40)

lo1.set_status(0)

try:
    for q_id in q_to_meas:
        pna.set_power(Qubits[q_id]['P'])
        pna.set_centerfreq(Qubits[q_id]['F'])
        pna.set_span(Qubits[q_id]['Span'])
        sweep.sweep(pna, (currents, current.set_current, 'Coil current'), filename = "anticrossing_p_{:.1f}dBm".format(pna.get_power()))
except KeyboardInterrupt:
    pass



Started at:  May 17 2017 15:49:50


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Coil current: 5.4e-05,	average cycle time:  7.8s	8s			
Elapsed time: 0 h 52 m 21.42 s


In [73]:
#Кубиты
q_id = 1

cur_step = 0.05e-3
Ipoints = np.arange( -Qubits[q_id]['Ispan']/2., Qubits[q_id]['Ispan']/2., cur_step)
currents = Ipoints + Qubits[q_id]['I0']
#currents = np.linspace( 0.5e-3, 2e-3, 20)

lo1.set_status(0)
#current.set_autorange(1)

pna.set_bandwidth(10)
pna.set_average(0)
pna.set_averages(1)

pna.set_nop(300)
pna.set_power(Qubits[q_id]['P'])
pna.set_centerfreq(Qubits[q_id]['F'])
pna.set_span(Qubits[q_id]['Span'])
current.set_status(1)

True

In [74]:
#Измерение
try:
    anti2D = sweep.sweep(pna, (currents, current.set_current, 'Coil current'), filename = "anticrossing")
except KeyboardInterrupt:  pass

Started at:  Jun 27 2018 15:29:43
Time left: 0 h 14 m 30.6 s,	parameter values: Coil current: -0.00075,	average cycle time:   27s		

In [ ]:
#Просто скан
currents = np.linspace( -50e-6, 50e-6, 200)
lo1.set_status(0)
current.set_autorange(1)

pna.set_bandwidth(100)
pna.set_average(0)
pna.set_averages(1)

pna.set_nop(1000)
current.set_status(1)

In [ ]:
#Измерение
try:
    anti2D = ps.S21vsPar(pna, currents, current.set_current, filename = "all_2" )
except KeyboardInterrupt:  pass

In [ ]:
#Зависимость частоты от тока
FvsI = dm.load_pkl("#3_FvsI")

In [ ]:
anti2D = dm.load_measurement('C:/qtlab/ReiData/data/Nov 16 2016/14-59-53/3/#3')

In [ ]:
FvsI= np.vstack(( anti2D.get_data()[0], np.zeros( len(anti2D.get_data()[0] ) )))
for i in range(0, len(anti2D.get_data()[0]) ):
    FvsI[1][i] = anti2D.get_data()[1][np.argmin(anti2D.get_data()[2][i])]

In [ ]:
plt.figure("fr")
plt.plot(FvsI[0],FvsI[1])

In [ ]:
dm.save_pkl_noplt("#5_FvsI",FvsI)

# Одномерный скан по второму тону

In [113]:
#Кубит
q_id = 1

pna.set_average(0)
pna.set_averages(1)
pna.set_power(Qubits[q_id]['P'])
pna.set_centerfreq(Qubits[q_id]['F'])
pna.set_nop(1)
pna.set_bandwidth(4)
lo1.set_power(Qubits[q_id]['Pex'])
lo1.set_status(1)

True

In [103]:
#current.set_autorange(1)
current.set_current(Qubits[q_id]['I0'])
#current.set_status(0)

True

In [ ]:
pna.set_power (-30)
lo1.set_power (10)
current.set_current(150e-6)

In [114]:
#Измерение
#ex_freqs = np.arange(6.9e9, 8.5e9, 1e6)
#ex_freqs = np.arange(3.155e9, 3.2e9, 5e6)
ex_span = 200e6
ex_freqs = Qubits[q_id]['F01'] + np.arange(-ex_span/2, ex_span/2 , 1e6)
lo1.set_status(1)

#Current = {:e}'''.format( lo1.get_power(), current.get_current() )
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "ex_{:.1f}dBm".format(lo1.get_power()))
except KeyboardInterrupt: pass 

Started at:  Jun 27 2018 18:04:42
Time left: 0 h 2 m 7.98 s,	parameter values: Second tone freq: 7.336e+09,	average cycle time: 0.71s		

In [ ]:
import plotting
import imp
imp.reload(plotting)
imp.reload(sweep)

In [ ]:
plot1 = plotting.plot_measurement(ex2D)

In [ ]:
plot[0].set_data([1,4,5,7], [0,8,9,0])

In [54]:
plt.figure('tmp_ph')
plt.plot(ex1D['S-parameter'][1][0], np.unwrap(np.angle(ex1D['S-parameter'][2])))
plt.figure('tmp_amp')
plt.plot(ex1D['S-parameter'][1][0], np.abs(ex1D['S-parameter'][2]))

In [51]:
ex1D['S-parameter'][1][0]

array([6.900e+09, 6.902e+09, 6.904e+09, ..., 9.994e+09, 9.996e+09,
       9.998e+09])

In [49]:
ex1D['S-parameter'][2]

array([-0.01077321-0.00210645j, -0.01081618-0.00286108j,
       -0.0118543 -0.00354977j, ..., -0.01076568-0.00283347j,
       -0.01098552-0.00190718j, -0.01132578-0.0034249 j])

# Двухтоновая спектроскопия

In [ ]:
pna.set_power(-10)
pna.set_nop(1)
pna.set_centerfreq(3.175e9)
#pna.set_centerfreq(3.176453e9)
#pna.set_centerfreq(7.6017351e9)
pna.set_bandwidth(200)

In [ ]:
current.set_current(150e-6)

In [ ]:
ex_freqs = np.arange(3.5e9, 4e9, 3e6)

In [ ]:
currents = np.arange( -54e-5, -52e-5, 1.5e-7)
#currents = np.arange( -50e-6, 350e-6, 30e-6)
#exc power
lo1.set_power(15)
#first tone power
pna.set_power(-10)
pna.set_bandwidth(5)
lo1.set_status(1)
sweep.sweep(pna, (currents, current.set_current, 'Coil current'), (ex_freqs, lo1.set_frequency, 'Exitation Frequency'),
            filename='Acouston_2_2tone_spectrum_-10dB_exc_15')
lo1.set_status(0)

In [ ]:
current.set_current(-209e-6)

In [ ]:
ex_freqs = np.arange(4.15e9, 4.5e9, 2e6)
ex_powers = np.arange(0, 18, 0.5)

pna.set_power(-20)
pna.set_bandwidth(3)
pna.set_average(1)
#pna.set_averages(2)
#lo1.set_power(-20)

#header = '''#Parameter =
#Ro power = {:f}'''.format( pna.get_power() )

lo1.set_status(1)
#ps.S21vs2DPar(pna, currents, current.set_current, ex_freqs, lo1.set_frequency, filename = "#2{:.1f}dBm".format(lo1.get_power()),\
#           header = header)
#ex2D = sweep.sweep(pna, (ex_powers, lo1.set_power, 'Second tone power'), (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "#2{:.1f}dBm".format(pna.get_power()),\
           #header = header)
ex2D = sweep.sweep(pna, (ex_powers, lo1.set_power, 'Second tone power'), (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "#2{:.1f}dBm".format(pna.get_power()))
lo1.set_status(0)

In [ ]:
plotting.plot_measurement(ex2D)

In [ ]:
plt.figure(figsize=(8,6))
plt.pcolormesh(ex2D.get_data()[3].T); plt.colorbar()

In [ ]:
plt.pcolormesh(ex2D.get_data()[4].T); plt.colorbar()

In [ ]:
ex_freqs = np.arange(3e9, 5e9, 0.5e9)


In [7]:
def current_setter(I):
    current.set_current(I)
    
    pna.set_sweep_mode("LIN")
    pna.set_nop(300)
    pna.set_centerfreq(Qubits[q_id]['F'])
    pna.set_span(Qubits[q_id]['Span'])
    pna.set_bandwidth(10)
    pna.set_power(Qubits[q_id]['P'])
    
    res_data = pna.measure()["S-parameter"]
    res_freqs = pna.get_freqpoints()
    center =  res_freqs[np.argmin(abs(res_data))]
    pna.set_nop(1)  
    pna.set_centerfreq( res_freqs[np.argmin(abs(res_data))] )
    pna.set_bandwidth(4)

In [88]:
current_setter(0)

6875332408.026756


In [8]:
#Померить все
#q_to_meas = Qubits.keys()
#Померить не все
q_to_meas = [1,]
#Шаг по втотрому тону
Fex_step = 1e6
Fex_offset = 50e6
Fex_start = 6.9e9

cur_step = 0.025e-3
#ex_freqs = np.arange(3e9, 3.35e9, 10e6)
#currents = np.arange( -0.9e-3, 0.9e-3, cur_step)

pna.set_nop(1)
pna.set_bandwidth(4)

current.set_current(0.)
current.set_status(1)
#current.set_autorange(1)

lo1.set_status(1)

#pna.set_timeout()


#root_dir = dm.mk_dir()

try:
    for q_id in q_to_meas:
        #end_dir = root_dir+"{:d}/".format(q_id)
        #dm.mk_dir(end_dir)
        
        ex_freqs = np.arange(Fex_start, Qubits[q_id]['F01']+Fex_offset, Fex_step)
        
        Ipoints = np.arange( -Qubits[q_id]['Ispan']/2., Qubits[q_id]['Ispan']/2., cur_step)
        
        currents = Ipoints + Qubits[q_id]['I0']
        
        pna.set_power(Qubits[q_id]['P'])
        pna.set_centerfreq(Qubits[q_id]['F'])
        
        lo1.set_power(Qubits[q_id]['Pex'])
        
        header = '#Ex power = {:f}'.format( lo1.get_power() )
        
        sweep.sweep(pna, (currents, current_setter, 'Coil current'), (ex_freqs, lo1.set_frequency, 'Exitation Frequency'),
            filename='2tone_spectrum')
        
except KeyboardInterrupt: pass

Started at:  Jun 27 2018 19:04:49
Time left: 0 h 0 m 0.0 s,	parameter values: Coil current: 0.000775,	Exitation Frequency: 7.467e+09,	average cycle time: 0.52s						
Elapsed time: 5 h 16 m 38.92 s


#Спектроскопия с учётм уползания резонатора

In [ ]:
q_id = 5
Fex_step = 4e6
Fex_offset = 150e6
Fex_start = 7e9
cur_step = 1e-6

ex_freqs = np.arange(Fex_start, Qubits[q_id]['F01']+Fex_offset, Fex_step)
        
Ipoints = np.arange( -Qubits[q_id]['Ispan']/2., Qubits[q_id]['Ispan']/2., cur_step)
        
currents = Ipoints + Qubits[q_id]['I0']
current.set_current(0)
current.set_status(1)
        
ro_freqs =  np.interp(currents, FvsI[0], FvsI[1])
pna.set_bandwidth(10)
pna.set_nop(1)
pna.set_power(Qubits[q_id]['P'])
#pna.set_centerfreq(Qubits[q_id]['F'])
        
lo1.set_power(Qubits[q_id]['Pex'])
lo1.set_status(1)
        
header = '#Ex power = {:f}'.format( lo1.get_power() )
try:
    ps.S21vs2DPar(pna, [currents, ro_freqs], [current.set_current, pna.set_centerfreq], ex_freqs, lo1.set_frequency,
              filename = "#{:d}".format(q_id), header = header)
except KeyboardInterrupt: pass    